In [ ]:
# Copyright 2025 Victor Semionov
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import os
from inspect import signature

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

from xlabml.datamodules import XLabDataModule
from xlabml import ROOT_DIR, CONF_DIR

sns.set_theme()

In [ ]:
os.chdir(ROOT_DIR)

In [ ]:
with open(CONF_DIR / 'defaults.yaml') as f:
    config = yaml.safe_load(f)
data_kwargs = {}
for parameter in signature(XLabDataModule.__init__).parameters:
    if parameter in config.get('data', {}):
        data_kwargs[parameter] = config['data'][parameter]

In [ ]:
datamodule = XLabDataModule(**data_kwargs)
datamodule.prepare_data()

In [ ]:
chunk_dataset = datamodule.datasets['train']
token_dataset = chunk_dataset.parent
text_dataset = token_dataset.parent

In [ ]:
len(token_dataset)

In [ ]:
np.random.seed(42)
sample_size = min(10_000, len(token_dataset))
sample_indices = np.random.permutation(sample_size)
sample = {**text_dataset.source[sample_indices], **token_dataset.dataset[sample_indices]}
df = pd.DataFrame(sample)

In [ ]:
df['length'] = df[token_dataset.column].apply(lambda indices: len(indices))

In [ ]:
df.head()

In [ ]:
df.length.mean(), df.length.std()

In [ ]:
ax = sns.histplot(data=df, x='length',  log_scale=True)
ax.set_xlabel('length')
ax.set_title('Distribution of text length in tokens')
plt.show()

In [ ]:
thresholds = [2**n for n in range(20)]
percentages = [sum(df.length > threshold) / len(df) for threshold in thresholds]
for threshold, percentage in zip(thresholds, percentages):
    print(f'Above {threshold}: {percentage * 100:.2f}%')